# AI Agents for Network Engineering

This (Jupyter Notebook) is an interactive coding environment that lets you write and run Python code in a web-based interface. Simply click on the play button or [SHIFT]+[ENTER] to execute the cell.

In [2]:
print("hello")

hello


## 1. Connectivity

### Catalyst Router (IOS XE)

In this lab we use the Catalyst 9000V IOS XE sandbox. Alternatively, you can also use the always-on sandbox.

1. Connect with your VPN client to the sandbox environment.
2. Open a terminal on the device and try to run a show-command to see if it works.
3. During the lab the credentials are stored in the **hosts.json** file to simulate external credentials retrieval.


**Hosts.json file content:**

In [ ]:
{
    "devnetsandboxiosxec9k.cisco.com" : {
            "type" : "cisco catalyst ios xe",
            "username" : "flpachin",
            "password" : "79QViiuKN5y_s3_1"
        },
    "10.100.80.2" : {
            "type" : "cisco catalyst ios xe",
            "username" : "admin",
            "password" : "Cisco123!"
        }
}

### Local Large Language Model (Ollama)

Let's see how the local large language model is working and the duration time of the prompt execution.

In [3]:
from ollama import chat
from ollama import ChatResponse

# LLM input = system prompt + user prompt
response: ChatResponse = chat(model = 'llama3.1', messages = [
    {
        'role': 'system',
        'content': 'You are a helpful networking assistant.',
    },
    {
        'role': 'user',
        'content': 'How can I show all routes on a Cisco switch?',
    }
])

# output
print(response['message']['content'])

To show all routes on a Cisco switch, you can use the `show ip route` command.

Here's how:

1. Connect to the Cisco switch using your preferred method (e.g., SSH, console cable).
2. Log in to the switch with your credentials.
3. Enter privileged EXEC mode by typing `enable`.
4. Enter global configuration mode by typing `configure terminal`.
5. Type `show ip route`.

This will display all routes on the switch, including both connected and static routes.

If you want to see just the routing table without any additional information, you can use the following syntax:

`show ip route | begin Gateway`

Or, if you're using a newer Cisco IOS version (15.2 or later), you can try:

`show ip route all`

Both of these commands will give you the same output as `show ip route`, but without any additional information.

Hope that helps!


## 2. Create Your First AI Agent with `smolagents`

Let's start writing our first AI agent in this framework and learn more about its components.

In [5]:
from smolagents.agents import CodeAgent
from smolagents import LiteLLMModel

model = LiteLLMModel(model_id="ollama/qwen2.5",
                     num_ctx=8192
                )

device_agent = CodeAgent(
                tools=[],
                model=model
)

device_agent.run("What does sh ip route do?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What does sh ip route do?                                                                                       │
│                                                                                                                 │
╰─ LiteLLMModel - ollama/qwen2.5 ─────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  print("The 'sh ip route' command is used to display the IP routing table on Cisco devices such as routers and    
  switches. This command helps administrators understand how the device forwards traffic for different             
  networks.")                                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The 'sh ip route' command is used to display the IP routing table on Cisco devices such as routers and switches. 
This command helps administrators understand how the device forwards traffic for different networks.

Out: None

[Step 1: Duration 17.78 seconds| Input tokens: 2,015 | Output tokens: 120]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("The 'sh ip route' command is used to display the IP routing table on Cisco devices such as         
  routers and switches. This command helps administrators understand how the device forwards traffic for           
  different networks.")                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: The 'sh ip route' command is used to display the IP routing table on Cisco devices such as 
routers and switches. This command helps administrators understand how the device forwards traffic for different 
networks.

[Step 2: Duration 7.34 seconds| Input tokens: 4,283 | Output tokens: 252]

"The 'sh ip route' command is used to display the IP routing table on Cisco devices such as routers and switches. This command helps administrators understand how the device forwards traffic for different networks."

## 3. Important Building Blocks for AI Agents

In order to understand AI agents, it is important to know these building blocks of AI agentic frameworks:

* Understanding the Agentic Behavior using OpenTelemetry
* Function/Tool Calling
* Structured Output

### 3.1 Using OpenTelemetry for Understanding & Debugging

Start the OpenTelemetry server, access it at [http://0.0.0.0:6006/projects/](http://0.0.0.0:6006/projects/) and run this code below. The next time you are executing a smolagent, you will see data.

In [ ]:
# ================== TELEMETRY ==================
from opentelemetry.sdk.trace import TracerProvider
from openinference.instrumentation.smolagents import SmolagentsInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
endpoint = "http://0.0.0.0:6006/v1/traces"
trace_provider = TracerProvider()
trace_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
SmolagentsInstrumentor().instrument(tracer_provider=trace_provider)

### 3.2. Function/Tool Calling with LLMs

Let's extend our code and define these custom functions. Execute the following code which does not provide any output.

In [ ]:
from smolagents import tool

@tool
def get_username_password_for_device(ip_address:str) -> str:
    """
    Returns the username and password separated by a comma for the given ip address or hostname.

    Args:
        ip_address: The IP address or hostname of the device

    Returns:
        str: A string with the username and password separated by a comma.
    """
    import json
    
    with open("hosts.json", 'r') as file:
        devices = json.load(file)
        return f"{devices[ip_address]['username']},{devices[ip_address]['password']}"

@tool
def show_ip_route(host:str,username:str,password:str,device_type:str = "cisco_ios",) -> str:
    """
    Return the routing table of the device. Executes the command 'show ip route'.

    Args:
        host: The IP address or hostname of the device
        username: The username for the device
        password: The password for the device
        device_type: The device type for the device

    Returns:
        str: The routing table of the device.
    """
    from netmiko import ConnectHandler    
                                                                                                          
    device = {                                                                                                                                                                 
        'ip': host,                                                                                                                                                   
        'username': username,                                                                                                                                                  
        'password': password,                                                                                                                                                  
        'device_type': device_type                                                                                        
    }                                                                                                                                                                          
                                                                                                                                                                                
    # Establish connection to the device                                                                                                                                       
    connection = ConnectHandler(**device)                                                                                                                                                                                                                                                                                     
                                                                                                                                                                                
    # Execute command to retrieve running configuration                                                                                                                        
    running_config_output = connection.send_command('show ip route')                                                                                                     
                                                                                                                                                                                
    return running_config_output

Now, let's add these functions to the CodeAgent and authorize to import these Python libraries in the sandboxed Python environment of the agent. Let's run this example with the following prompt.

In [ ]:
from smolagents.agents import CodeAgent
from smolagents import LiteLLMModel

model = LiteLLMModel(model_id="ollama/qwen2.5",
                     num_ctx=8192)

device_agent = CodeAgent(tools=[get_username_password_for_device,
                         show_ip_route],
                  model=model,
                  additional_authorized_imports=['ncclient', 'netmiko','requests','paramiko','io'],
                 )
# give the sandboxed Python interpreter access to read/write files outside (use with caution!)
device_agent.python_executor.static_tools["open"] = open

# Routing Table Summary as Markdown
device_agent.run("""
                 Summarize the routing table on the Cisco device 10.10.20.48.
                 Save it in the Markdown file 'routing_table_summary.md'.
                 You will at first need the username and password for the device.
                 """)

**Expected results**: After execution, the LLM intelligently calls `get_username_password_for_device` first and then `show_running_configuration` based on your prompt. You should see the summary of the routing table in the newly created file `routing_table_summary.md` which should be located in our project folder.

### 3.3 Structured Output with LLMs

Let's understand structured outputs with the following code. At first, we define just one function which returns a show command on the device. Execute this cell, there should be no output.

In [ ]:
def run_ios_show_command_on_device_trusted(show_command:str,host:str) -> str:
    """
    Returns the output of the provided show command from the device with the given host, username, password, and device type.

    Args:
        show_command: The show command to run on the device
        host: The IP address or hostname of the device
        device_type: The device type for the device

    Returns:
        str: Output of the provided show command
    """
    print("Running show command on device...")

    from netmiko import ConnectHandler
    import json

    with open("hosts.json", 'r') as file:
        devices = json.load(file)
        username = devices[host]["username"]
        password = devices[host]["password"]
                                                          
    device = {                                                                                                                                                                 
        'ip': host,                                                                                                                                                   
        'username': username,                                                                                                                                                  
        'password': password,                                                                                                                                                  
        'device_type': "cisco_ios",                                                                                        
    }                                                                                                                                                                          
                                                                                                                                                                                
    # Establish connection to the device                                                                                                                                       
    connection = ConnectHandler(**device)

    if show_command.startswith("show") or show_command.startswith("sh"):
        # Execute command to retrieve running configuration                                                                                                                        
        running_config_output = connection.send_command(show_command)
        print(running_config_output)
        return running_config_output
    else:
        return "Error! You are only allowed to run show commands. Try again and use a show command."

Then, let's define the output schema at first, the parameters for our prompt:

In [ ]:
from pydantic import BaseModel

# here we define the model for the response
class Cat8000(BaseModel):
    ios_version: str
    configuration_register: str

# define your parameters
ios_show_command = "show version"
device_host = "10.10.20.48" # or "devnetsandboxiosxe.cisco.com"

Then let's define our prompts: system + user prompt and tell the LLM how to structure the output and run the query:

In [ ]:
from ollama import chat

response = chat(
    messages=[
        {
        'role': 'system',
        'content': 'You are a helpful networking assistant.',
    },
    {
        'role': 'user',
        'content': f'''What can you say about my Cisco switch?
                    Here is the output of the "show version" command:
                    {run_ios_show_command_on_device_trusted(ios_show_command,device_host)}''',
    }
    ],
    model='llama3.1',
    format=Cat8000.model_json_schema(),
)

cat8000_instance = Cat8000.model_validate_json(response.message.content)

Finally, we can access each of our defined data points:

In [ ]:
print("LLM response output according to your schema:")
print(f"IOS Version: {cat8000_instance.ios_version}")
print(f"Config Register: {cat8000_instance.configuration_register}")

You will see that the LLM is extracting information from the `show version` command according to your define schema.

# Next steps

Check out the use-cases which are stored in the folder **complete examples**! Run them and go through the code to understand the concept and how you can apply them to your environment and use-cases.